# Ames, Iowa Housing Data - Exploration and Preparation

In this notebook, we conduct our exploratory data analysis (EDA) and prepare our data for modeling using linear regression. This notebook contains the following sections:
- [Exploratory Data Analysis (EDA)](#Exploratory-Data-Analysis-(EDA))
- [Data Preparation](#Data-Preparation)

In [1]:
# Import relevant libraries

import pandas as pd
import seaborn as sns
from general_functions import LeeFunctions as lf

In [2]:
# Read in housing data files 

orig = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

## Exploratory Data Analysis (EDA)

In this section, we look at what information is contained in our data sets, including data types, feature names and rudimentary statistics. From this, we determine which numerical variables are most closely correlated with our target variable, sale price. We use this exploration to select a group of features which we believe have the most predictive power, which will be used to create our final independent variable matrix

In [3]:
# Snake_case column names for ease of use. Create a new data feature [age] from the year sold and year built/remodeled features
for frames in [orig,test]:
    frames.columns = [col.lower().replace(' ','_') for col in frames.columns]
    frames['age'] = frames['yr_sold'] - frames['year_remod/add']

In [4]:
# Take a look at the first few rows of the data set, just to see what we're dealing with

print('Training data set size:',orig.shape)
orig.head()

Training data set size: (2051, 82)


,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice,age
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,130500,5
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,4,2009,WD,220000,12
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,1,2010,WD,109000,3
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,174000,3
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,138500,17


In [6]:
# Determine the number of null values in the data set. These features will have to have their null values removed or imputed later if they are to be included in the model

lf.check_frame(orig)

lot_frontage: 330 null values
alley: 1911 null values
mas_vnr_type: 22 null values
mas_vnr_area: 22 null values
bsmt_qual: 55 null values
bsmt_cond: 55 null values
bsmt_exposure: 58 null values
bsmtfin_type_1: 55 null values
bsmtfin_sf_1: 1 null values
bsmtfin_type_2: 56 null values
bsmtfin_sf_2: 1 null values
bsmt_unf_sf: 1 null values
total_bsmt_sf: 1 null values
bsmt_full_bath: 2 null values
bsmt_half_bath: 2 null values
fireplace_qu: 1000 null values
garage_type: 113 null values
garage_yr_blt: 114 null values
garage_finish: 114 null values
garage_cars: 1 null values
garage_area: 1 null values
garage_qual: 114 null values
garage_cond: 114 null values
pool_qc: 2042 null values
fence: 1651 null values
misc_feature: 1986 null values


In [7]:
# Find correlations among numerical values in comparison to our target variable, sales price, and show the top 10 features

salecor = orig.corr()[['saleprice']].sort_values(by='saleprice', axis=0)
print(salecor[-11:-1])

                saleprice
garage_yr_blt    0.533922
full_bath        0.537969
year_remod/add   0.550370
year_built       0.571849
1st_flr_sf       0.618486
total_bsmt_sf    0.628925
garage_cars      0.648220
garage_area      0.650270
gr_liv_area      0.697038
overall_qual     0.800207


From our correlation analysis, we selected a group of variables that we believe will predict sale price well. Certain groups of features were represented by a single variable due to the high level of dependency between them. For example, only garage area was selected, since the number of cars the garage could hold would likely be highly correlated to the size. From this, we selected the following numerical values: 
- Above ground living area (gr_liv_area)
- Overall quality (overall_qual)
- Age
- Total basement area (total_bsmt_sf)
- Garage area

### Removing Outliers

In [7]:
%%capture
# Create seaborn pairplot of our numerical variables against sale price in order to see how these relationships look in isolation. This helps us identify outliers in each category

sns.pairplot(data=orig, y_vars=['saleprice'], x_vars=['gr_liv_area','overall_qual','age','total_bsmt_sf','garage_area','totrms_abvgrd']);

![](../images/sns_pairplot.png)

From the pairplot, we identified several potential outliers (circled in red). As suggested by the [original data set description](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt), we believe removing houses with total living areas greater than 4,000 sq. ft. will not adversely affect our model. We apply a similar rationale for removing the data points with extremely large total basement areas.

In [8]:
# Remove outlier data points with extremely large living and basement areas. Once these rows have been dropped, we can then create our data sets for model fitting

orig.drop(orig[orig['gr_liv_area'] > 4000].index, inplace=True)
orig.drop(orig[orig['total_bsmt_sf'] > 4000].index, inplace=True)

For non-numerical values, we chose two features based on our general experience that we believe have the greatest impact on the value of a house: **location** and the **type** of house. These correspond to the features _neighborhood_ and *bldg_type*, respectively

## Data Preparation
To use our selected data to create a regression model, we first have to make sure the data is cleaned and organized such that the fit can be applied. Our first step was to organize the selected features into several categories, as described below

|Name|Description|
|---|---|
nums|Variables whose values must be numeric, either int or float. Dictionary includes the minimum and maximum expected values for the variable
cats|Variables that are categorical or ordinal in nature. Types can be both numerical or strings. Variables in this category default to have missing values imputed to the mode of the variable
median|Variables where it is decided that the proper imputation method is by replacing null values with the training data median. This is for variables where having a zero value would be illogical, such as having a house with 0 sq. ft. of living area
blanks|Variables where it is decided that it is acceptable to have a 0 value. For example, it is possible for a house to have a 0 garage area (no garage)
feature|Total feature list, which is the set union of the cats and nums lists

The data sets were then cleaned of null values by imputing missing values using the methods described in each of the categories above. Features in the nums category were checked to ensure they contained only numeric values and that these values fell within accepted ranges.

In [9]:
# Organize variables as described above. Create model data sets which contains only our variables of interest, as well as our target
# vector (orig['saleprice']), and wrap in a list to be passed to our modeling notebook later

nums = {'overall_qual':[1,10],
        'gr_liv_area':[0,'none'],
        'age':[0,300],
        'total_bsmt_sf':[0,'none'],
        'garage_area':[0,'none']}

cats = ['bldg_type', 'neighborhood', 'overall_qual']
median = ['age','gr_liv_area']
blanks = ['total_bsmt_sf','garage_area']
features = set(list(nums) + cats)

datasets = [orig.copy()[features], test.copy()[features], orig['saleprice']]

### Data Imputation

In [10]:
# Clean our data sets to impute missing values and check to make sure our features are of the correct type so that we don't run into errors during the modeling process. Corrections
# (or lack thereof) are printed as outputs. For a more detailed explanation of each of the functions called here, consult the documentation in the general function .py file
# Create dummy columns for categorical variables

for x in range(2):
    frames = datasets[x]
    print(f'Frame {x}:')
    lf.impute_frame(frames, modes=cats, medians=median, zeroes=blanks, reference=datasets[0])
    lf.check_frame(frames, numericals = nums)
    lf.check_range(frames, numericals = nums, correct=True)
    lf.collapse_ordinal(frames, 'overall_qual', [1,10], 5)
    frames = pd.get_dummies(frames, columns=cats, drop_first=True)
    datasets[x] = frames    
    print('\n')

Frame 0:
Feature [total_bsmt_sf]: 1 null values converted to 0
Feature [garage_area]: 1 null values converted to 0
No null values in data set
Checking for numerical type mismatches in features: ['overall_qual', 'gr_liv_area', 'age', 'total_bsmt_sf', 'garage_area']
No type mismatches found
Checking for out of range values in features: ['overall_qual', 'gr_liv_area', 'age', 'total_bsmt_sf', 'garage_area']
Feature [age] minimum values corrected


Frame 1:
No null values in data set
Checking for numerical type mismatches in features: ['overall_qual', 'gr_liv_area', 'age', 'total_bsmt_sf', 'garage_area']
No type mismatches found
Checking for out of range values in features: ['overall_qual', 'gr_liv_area', 'age', 'total_bsmt_sf', 'garage_area']
No out of range values in selected features




In [11]:
# When creating dummy columns, if values are missing in one of the data sets, it's possible to not create the same number of columns in one of the data sets. This function checks
# the columns of both the test and training data and ensures that they are the same and aligned

datasets[0], datasets[1] = lf.match_columns(datasets[0], datasets[1])

In [12]:
%%capture
%store datasets

# Store our cleaned and prepped data sets to pass to our modeling notebook